# Train the model to identify the category of cloth

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers, Sequential, optimizers, callbacks, models, backend
from keras.preprocessing.image import ImageDataGenerator

import os
import pandas as pd

2022-07-01 16:32:42.708899: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sahil/myfiles/auv/vslam/orb_slam/devel/lib:/home/sahil/myfiles/auv/tarang_ws/devel/lib:/home/sahil/myfiles/auv/training/assing_1/work/devel/lib:/opt/ros/noetic/lib
2022-07-01 16:32:42.708938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Load the data
data_df = pd.read_csv('../dataset/anno/img_label2.csv',sep=',')
data_df['category_label'] = data_df['category_label'].astype(str)
data_df.head()

,image_name,category_label
0,../dataset/img/Cities_Graphic_Boxy_Tee/img_000...,6
1,../dataset/img/Pleated_A-Line_Dress/img_000000...,8
2,../dataset/img/Longline_Denim_Jacket/img_00000...,3
3,../dataset/img/Fuzzy_Plaid_Cardigan/img_000000...,1
4,../dataset/img/V-Neck_Tank/img_00000053.jpg,5


In [3]:
# construct the data generators
input_size = (224,224)
num_class = 11

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255, rotation_range=20, zoom_range=0.2)

train_data_gen = datagen.flow_from_dataframe(
    data_df, x_col='image_name',y_col = 'category_label',
    subset='training',target_size=input_size,directory='.',
    class_mode='categorical',batch_size=32, seed=42
)

val_data_gen = datagen.flow_from_dataframe(
    data_df, x_col='image_name',y_col = 'category_label',
    subset='validation',target_size=input_size,directory='.',
    class_mode='categorical',batch_size=32, seed=42
)

Found 51845 validated image filenames belonging to 11 classes.
Found 12961 validated image filenames belonging to 11 classes.


In [5]:
# get label mapping
label_map = train_data_gen.class_indices
print(label_map)
label_map = [0,1,10,2,3,4,5,6,7,8,9]

{'0': 0,
 '1': 1,
 '10': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [ ]:
# choose the base model, Xception model is choosen here
encoder = keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in encoder.layers:
    layer.trainable = True
print(encoder.output_shape)
print(len(encoder.layers))
keras.utils.plot_model(encoder, show_shapes=True, show_layer_names=True)


In [16]:
# build a classifier
inp_classifier = Sequential([

    layers.Input(shape = encoder.output_shape[1:]),
    layers.GlobalAveragePooling2D(),

    layers.Dense(512, activation='relu', kernel_regularizer = regularizers.l2(1e-5)),
    # layers.Dropout(0.1),
    
    layers.Dense(128, activation='relu', kernel_regularizer = regularizers.l2(1e-7)),
    layers.Dropout(0.1),
    
    layers.Dense(num_class, activation='softmax')
    
], name='classifier' )

inp_classifier.summary()


Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_8 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 11)                1419      
Total params: 1,116,171
Trainable params: 1,116,171
Non-trainable params: 0
_________________________________________________________________


In [17]:
# stack the encoder and classifier to create the model
model = tf.keras.Model(inputs=encoder.input, outputs=inp_classifier(encoder.output),name = 'complete_model')
model.count_params()

24680971

In [18]:
# compile the model
model_save_path = '/content/drive/MyDrive/hopnob/model_v7_0t'
ckp = callbacks.ModelCheckpoint(filepath=model_save_path, save_best_only=True, monitor='val_accuracy')

opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# train the mdel
model.fit(train_data_gen, epochs=10, validation_data=val_data_gen, callbacks=[ckp])

I got nearly 65% percent accuracy on the validation set using this model